In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import functions_strava as fs

<h1>Tables of Contents<span class="tocSkip"></span></h1>
<div class="toc">
  <ul class="toc-item">
    <li>
      <span><a href="#1.-Open-and-save-file" data-toc-modified-id="1.-Open-and-save-file-0.1">1. Open and save file</a></span>
    </li>
    <li>
      <span><a href="#2.-Retrieve-activity-ID" data-toc-modified-id="2.-Retrieve-activity-ID-1">2. Retrieve activity ID</a></span>
      <ul class="toc-item">
        <li><span><a href="#2.1-Login-to-Strava" data-toc-modified-id="2.1-Login-to-Strava-1.1">2.1 Login to Strava</a></span></li>
        <li><span><a href="#2.2-Loop-on-all-week-of-the-interval" data-toc-modified-id="2.2-Loop-on-all-week-of-the-interval-1.2">2.2 Loop on all week of the interval</a></span></li>
        <li><span><a href="#2.3-Dealing-with-HT-activity-and-manually-input-activity" data-toc-modified-id="2.3-Dealing-with-HT-activity-and-manually-input-activity-1.3">2.3 Dealing with HT activity and manually input activity</a></span></li>
      </ul>
    </li>
    <li>
      <span><a href="#3.-Retrieve-activity-data" data-toc-modified-id="3.-Retrieve-activity-data-2">3. Retrieve activity data</a></span>
      <ul class="toc-item">
        <li><span><a href="#3.1-Loop-to-retrieve-activity-data" data-toc-modified-id="3.1-Loop-to-retrieve-activity-data-2.1">3.1 Loop to retrieve activity data</a></span></li>
        <li><span><a href="#3.2-Loop-to-retrieve-activity-date-and-type" data-toc-modified-id="3.2-Loop-to-retrieve-activity-date-and-type-2.2">3.2 Loop to retrieve activity date and type</a></span></li>
      </ul>
    </li>
    <li><span><a href="#4.-Deleting-the-data-when-not-moving-and-useless-field" data-toc-modified-id="4.-Deleting-the-data-when-not-moving-and-useless-field-3">4. Deleting the data when not moving and useless field</a></span></li>
  </ul>
</div>

## 1. Open and save file 

In [ ]:
#save list
fs.save_list(List_ID, "List_ID_201201_201219");

In [ ]:
#load list
List_ID = fs.load_list("List_ID_201201_201219");
len(List_ID)

In [ ]:
#save dataset
fs.save_dataset(Dataset);

In [ ]:
#load dataset
Dataset = fs.load_dataset();
len(Dataset)

To limit the size of the file and avoid too much latency you can use several datafile

In [ ]:
# Define the filename for the JSON file
filename = "Dataset_training_2013_2018.json"

# Open the JSON file and load its contents into a dictionary
with open(filename, "r") as f:
    Dataset_13_18 = json.load(f)


In [ ]:
# Define the filename for the JSON file
filename = "Dataset_training_Manual.json"

# Save the dictionary as a JSON file
with open(filename, "w") as f:
    json.dump(Dataset_training_Manual, f)

# 2. Retrieve activity ID

## 2.1 Login to Strava

To acces and recover all the public data of strava, we first need to login :

In [ ]:
driver = fs.strava_login_selen();

## 2.2 Loop on all week of the interval

In [ ]:
#Define the starting point and ending point of the loop
start = 201201
end = 201219

interval = fs.interval_loop(start,end)

In [ ]:
#Retrieve activity ID 
List_ID = fs.get_activity_ID(interval, driver)
len(List_ID)

In [ ]:
List_ID

## 2.3 Dealing with HT activity and manually input activity

In [ ]:
# Add Home trainer activity
List_ID_HT=["147426850","667135113","1104286349","1500094060","2926777241","2926777163","3291863486","3291865260","3527747185","3564333412","3559969298","4596814497","4692053243","4692053249","5540126838","5749870577","5745660704","6443383825","7398671957","8669736110","8818255598","9020996966"]

In [ ]:
# Add manually input activity
List_ID_Manual=["283039268","386635246","430450073","482259029","608203721","608204685","670797424","686858551","720033312","833112424","841142478","839894491","839894291","933637680","959102402","993065367","999077157","1003849466","1014854217","1176432990","1271237120","1395932601","1503881192","1503879445","1503877605","1985361995","1985361646","1985361504","3122368544","3184001613","3209030117","3810865502","3872485004","5111137080","5285673554","5426371236","5565025517","6267914197","7037356576","8398585701"]

In [ ]:
#Dataset_training_HT={}

In [ ]:
#Dataset_training_Manual={}

In [ ]:
#Retrieve home trainer activity data
def get_activity_HT_data(List_ID_Load,Dataset,driver):
    # Loop for the outdoor ride 
    for ID in List_ID_Load : 
        if ID in Dataset :
            time.sleep(0.01)
        else :
            print(ID)
            # Open new window which contain the activity data
            driver.switch_to.window(driver.window_handles[0])
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(0.02)
            driver.get(f"https://www.strava.com/activities/{ID}/streams?stream_types%5B%5D=time&stream_types%5B%5D=heartrate&stream_types%5B%5D=altitude&stream_types%5B%5D=distance&stream_types%5B%5D=moving&stream_types%5B%5D=velocity_smooth&stream_types%5B%5D=cadence&stream_types%5B%5D=latlng&stream_types%5B%5D=watts&stream_types%5B%5D=watts_calc&stream_types%5B%5D=grade_adjusted_distance&stream_types%5B%5D=temp&stream_types%5B%5D=grade_smooth")
            time.sleep(0.5)
            # Recover all the content
            activity_raw=driver.page_source
    
            # Keep only the useful infos
            activity_clean=activity_raw.split('">')[2].split('<')[0]
    
            # Convert them as dictionnary
            activity_dict = json.loads(activity_clean)
    
            # Add them to the dataset
            Dataset[ID]=activity_dict
            driver.execute_script("window.close();")
        time.sleep(0.4)
    return Dataset

Dataset_training_HT = get_activity_HT_data(List_ID_HT,Dataset_training_HT,driver)

In [ ]:
#Retrieve home trainer activity type and date :

def get_activity_HT_type_date(session,Dataset):
    for ID in Dataset : 
        if "date" in Dataset[ID] :
            time.sleep(0.01) 
        else :
            
            # Fetch the web page
            url = f"https://www.strava.com/activities/{ID}"

            # Send a GET request to the URL and retrieve the HTML content
            response = session.get(url)
            print(response)
            time.sleep(1)
            html_content = response.content

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            print(ID)
            # Recover the date of the activity
            time_element = soup.find("time")
            time_text = time_element.get_text(strip=True)

            # Convert time string to datetime object
            try:
                # Try parsing with format '%I:%M %p on %A, %B %d, %Y'
                datetime_obj = datetime.strptime(time_text, '%I:%M %p on %A, %B %d, %Y')
            except ValueError:
                try:
                    # Try parsing with format '%A, %B %d, %Y'
                    datetime_obj = datetime.strptime(time_text, '%A, %B %d, %Y')
                except ValueError:
                    return None
            #datetime_obj = datetime.strptime(time_text, '%I:%M %p on %A, %B %d, %Y')
            #datetime_obj = datetime.strptime(time_text, '%A, %B %d, %Y')

            # Format datetime object to desired format
            date_clean = datetime_obj.strftime('%d/%m/%Y')

            time.sleep(0.3)
    
            # Recover the type of activity
            type_raw = soup.find('title')
            
            type_clean = type_raw.string.split('| ')[1].split(' |')[0]
    
            # Add the date and type to the correspnding activity in the dataset
            Dataset[ID]["date"]=date_clean
            Dataset[ID]["type"]=type_clean
    return Dataset


Dataset_training_HT = get_activity_HT_type_date(session,Dataset_training_HT )

In [ ]:
#Retrieve manually input activity type and date, distance and duration

def get_activity_manual_type_date(List_ID_Load,session,Dataset):
    for ID in List_ID_Load :
        if ID in Dataset :
            time.sleep(0.01) 
        else :
            
            # Fetch the web page
            url = f"https://www.strava.com/activities/{ID}"

            # Send a GET request to the URL and retrieve the HTML content
            response = session.get(url)
            print(response)
            time.sleep(1)
            html_content = response.content

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            print(ID)
            # Recover the distance of the activity
            kilometer_element = soup.find('abbr', {'title': 'kilometers'})
            kilometer_clean = kilometer_element.previous_sibling.strip()

            # Recover the duration of the activity
             
            duration_element = soup.find('div', class_='label', string='Duration')
            if duration_element is None:
                divs = soup.find_all('div', class_='label')
                duration_value = divs[1].find_previous_sibling('strong').text.strip()
            else :
                duration_value = duration_element.find_previous_sibling('strong').text.strip()

            # Check the number of colons in the time string
            if duration_value.count(':') == 2:
                # If there are two colons, parse with format '%H:%M:%S'
                duration_timedelta = datetime.strptime(duration_value, '%H:%M:%S')
            else:
                # If there is one colon, parse with format '%M:%S'
                duration_timedelta = datetime.strptime(duration_value, '%M:%S')

            # Format the duration as hh:mm:ss
            duration_clean = duration_timedelta.strftime('%H:%M:%S')

        
            # Recover the date of the activity
            time_element = soup.find("time")
            time_text = time_element.get_text(strip=True)
            
            # Convert time string to datetime object
            try:
                # Try parsing with format '%I:%M %p on %A, %B %d, %Y'
                datetime_obj = datetime.strptime(time_text, '%I:%M %p on %A, %B %d, %Y')
            except ValueError:
                try:
                    # Try parsing with format '%A, %B %d, %Y'
                    datetime_obj = datetime.strptime(time_text, '%A, %B %d, %Y')
                except ValueError:
                    return None
            #datetime_obj = datetime.strptime(time_text, '%I:%M %p on %A, %B %d, %Y')
            #datetime_obj = datetime.strptime(time_text, '%A, %B %d, %Y')

            # Format datetime object to desired format
            date_clean = datetime_obj.strftime('%d/%m/%Y')

            time.sleep(0.3)
            # Add a new key-value pair
            Dataset[ID] = {}
            # Recover the type of activity
            type_raw = soup.find('title')
            
            type_clean = type_raw.string.split('| ')[1].split(' |')[0]
    
            # Add the date and type to the correspnding activity in the dataset
            Dataset[ID]["date"]=date_clean
            Dataset[ID]["type"]=type_clean
            Dataset[ID]["Distance (Km)"]=kilometer_clean
            Dataset[ID]["Duration (hh:mm:ss)"]=duration_clean
    return Dataset


Dataset_training_Manual = get_activity_manual_type_date(List_ID_Manual,session,Dataset_training_Manual)

# 3. Retrieve activity data

## 3.1 Loop to retrieve activity data

In [ ]:
# Retrieving activity data and store it in the dataset
Dataset = fs.get_activity_data(List_ID,Dataset,driver)

In [ ]:
# check value of Dataset after retrieving data
len(Dataset)

## 3.2 Loop to retrieve activity date and type

In [ ]:
#We need to sign in in order too make several request without having error 429
session = requests.Session()
login_response = session.get('https://strava.com/login')
soup = BeautifulSoup(login_response.text)
token = soup.find('input', {'name': 'authenticity_token'}).get('value')
print(f"Token {token}")
response = session.post('https://www.strava.com/session', data={'email':'violeau.p@gmail.com', 'password':'Tabu6415', 'authenticity_token': token})
print(response)

In [ ]:
#Retrieve date and type of activity and had them to the dataset
Dataset = fs.get_activity_type_date(session,Dataset_13_18)

In [ ]:
def get_activity_type_date(session,Dataset):
    for ID in Dataset : 
        if "date" in Dataset[ID] :
            time.sleep(0.01) 
        else :
           # Fetch the web page
            url = f"https://www.strava.com/activities/{ID}"

            # Send a GET request to the URL and retrieve the HTML content
            response = session.get(url)
            print(response)
            time.sleep(1)
            html_content = response.content

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            print(ID)
            # Recover the distance of the activity
            kilometer_element = soup.find('abbr', {'title': 'kilometers'})
            kilometer_clean = kilometer_element.previous_sibling.strip()

            # Recover the duration of the activity
             
            duration_element = soup.find('div', class_='label', string='Duration')
            if duration_element is None:
                divs = soup.find_all('div', class_='label')
                duration_value = divs[1].find_previous_sibling('strong').text.strip()
            else :
                duration_value = duration_element.find_previous_sibling('strong').text.strip()

            # Check the number of colons in the time string
            if duration_value.count(':') == 2:
                # If there are two colons, parse with format '%H:%M:%S'
                duration_timedelta = datetime.strptime(duration_value, '%H:%M:%S')
            else:
                # If there is one colon, parse with format '%M:%S'
                duration_timedelta = datetime.strptime(duration_value, '%M:%S')

            # Format the duration as hh:mm:ss
            duration_clean = duration_timedelta.strftime('%H:%M:%S')

        
            # Recover the date of the activity
            time_element = soup.find("time")
            time_text = time_element.get_text(strip=True)
            
            # Convert time string to datetime object
            try:
                # Try parsing with format '%I:%M %p on %A, %B %d, %Y'
                datetime_obj = datetime.strptime(time_text, '%I:%M %p on %A, %B %d, %Y')
            except ValueError:
                try:
                    # Try parsing with format '%A, %B %d, %Y'
                    datetime_obj = datetime.strptime(time_text, '%A, %B %d, %Y')
                except ValueError:
                    return None
            #datetime_obj = datetime.strptime(time_text, '%I:%M %p on %A, %B %d, %Y')
            #datetime_obj = datetime.strptime(time_text, '%A, %B %d, %Y')

            # Format datetime object to desired format
            date_clean = datetime_obj.strftime('%d/%m/%Y')

            time.sleep(0.3)
            # Add a new key-value pair
            Dataset[ID] = {}
            # Recover the type of activity
            type_raw = soup.find('title')
            
            type_clean = type_raw.string.split('| ')[1].split(' |')[0]
    
            # Add the date and type to the correspnding activity in the dataset
            Dataset[ID]["date"]=date_clean
            Dataset[ID]["type"]=type_clean
            Dataset[ID]["Distance (Km)"]=kilometer_clean
            Dataset[ID]["Duration (hh:mm:ss)"]=duration_clean
    return Dataset


Dataset = get_activity_type_date(session,Dataset)

# 4. Deleting the data when not moving and useless field

In [ ]:
for ID in Dataset : 
        if "watts_calc" in Dataset[ID] :
            del Dataset[ID]["watts_calc"]
        else :
            continue

In [ ]:
for ID in Dataset : 
        if "grade_smooth" in Dataset[ID] :
            del Dataset[ID]["grade_smooth"]
        else :
            continue

In [ ]:
for ID in Dataset : 
    if  "moving" in Dataset[ID] :
        for data_type in Dataset[ID]:
            if (data_type != "moving") and (data_type != "date") and (data_type != "type"):
                if len(Dataset[ID]["moving"]) == len(Dataset[ID][data_type]) :
                    Dataset[ID][data_type] = [Dataset[ID][data_type][i] for i in range(len(Dataset[ID]["moving"])) if Dataset[ID]["moving"][i]]
                else :
                    continue
            else :
                continue
    else :
        continue

In [ ]:
for ID in Dataset : 
    if  "moving" in Dataset[ID] :
        if len(Dataset[ID]["moving"]) != len(Dataset[ID]["time"]) :
            del Dataset[ID]["moving"]
        else:
            continue
    else :
        continue